In [1]:
%matplotlib inline
import torch
import matplotlib.pyplot as plt
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

In [2]:
def synthetic_data(w, b, num_examples):
    """生成y=Xw+b+噪声"""
    X = torch.normal(0, 1, (num_examples, len(w)))
    y = torch.matmul(X, w) + b
    y += torch.normal(0, 0.01, y.shape)
    return X, y.reshape((-1, 1))

In [3]:
true_w = torch.tensor([2, -3.4])
true_b = 4.2
features, labels = synthetic_data(true_w, true_b, 1000)

In [4]:
from torch.utils import data

def load_array(data_arrays, batch_size, is_train=True):
    """构造一个PyTorch数据迭代器"""
    dataset = data.TensorDataset(*data_arrays)
    return data.DataLoader(dataset, batch_size, shuffle=is_train)

batch_size = 10
data_iter = load_array((features, labels), batch_size)

In [5]:
next(iter(data_iter))

[tensor([[ 1.1959, -0.8664],
         [ 0.1036, -0.2063],
         [ 0.2577,  0.7825],
         [ 1.4716,  1.4875],
         [-0.3767,  0.1074],
         [ 0.6863, -0.0514],
         [ 0.5423, -1.0115],
         [-0.1717,  0.6440],
         [ 0.4567,  0.9461],
         [ 0.1699,  0.7582]]),
 tensor([[9.5417],
         [5.1131],
         [2.0497],
         [2.0713],
         [3.1000],
         [5.7493],
         [8.7301],
         [1.6715],
         [1.9108],
         [1.9608]])]

In [6]:
from torch import nn
net = nn.Sequential(nn.Linear(2, 1))

In [7]:
net[0].weight.data.normal_(0, 0.01)
net[0].bias.data.fill_(0)

tensor([0.])

In [8]:
loss = nn.MSELoss()

In [9]:
trainer = torch.optim.SGD(net.parameters(), lr=0.03)

In [10]:
num_epochs = 3
for epoch in range(num_epochs):
    for X, y in data_iter:
        l = loss(net(X) ,y)
        trainer.zero_grad()
        l.backward()
        trainer.step()
    l = loss(net(features), labels)
    print(f'epoch {epoch + 1}, loss {l:f}')

epoch 1, loss 0.000283
epoch 2, loss 0.000107
epoch 3, loss 0.000109


In [11]:
w = net[0].weight.data
b = net[0].bias.data
print('w的估计误差：', true_w - w.reshape(true_w.shape))
print('b的估计误差：', true_b - b)

w的估计误差： tensor([-0.0006, -0.0015])
b的估计误差： tensor([0.0007])
